## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

rureviews = pd.read_csv("drive/MyDrive/Colab Notebooks/women-clothing-accessories.csv", sep='\t')

Mounted at /content/drive


Применим полученные навыки и решим задачу анализа тональности отзывов.

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор.

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer() # лемматизатор

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 56.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=6634a8f53af0c2e0195936528963ff213c735217ca1e3138a4c481b3c170fcad
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
# Токенизация
import nltk
from nltk.tokenize import word_tokenize # готовый токенизатор библиотеки nltk
nltk.download('stopwords') # импорт стоп-слов
from nltk.corpus import stopwords
nltk.download('punkt')
import string

stop_words = stopwords.words('russian') # стоп-слова для русского языка

def preprocess(text):
    tokens = word_tokenize(text.lower()) # приведение к нижнему регистру
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation] # удаление стоп-слов и ненужной пунктуации
    tokens = [pymorphy2_analyzer.parse(token)[0].normal_form for token in tokens] # лемматизация

    return " ".join(tokens)


rureviews['review_preprocessed'] = rureviews['review'].apply(preprocess)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB # наивный байесовский классификатор

x_train, x_test, y_train, y_test = train_test_split(rureviews.review_preprocessed, rureviews.sentiment, train_size = 0.7)

In [ ]:
# Настройка векторайзеров

# Мешок n-грамм
from sklearn.feature_extraction.text import CountVectorizer

def select_ngram_range(corpus, result, min_n, max_n):
    best_ngram_range = (min_n, min_n)
    best_score = 0
    for n in range(min_n, max_n+1):
        for m in range(n+1, max_n+1):
            vectorizer = CountVectorizer(ngram_range=(n, m))
            X = vectorizer.fit_transform(corpus)
            bayes = MultinomialNB().fit(X, result)
            prediction = bayes.predict(X)
            score = accuracy_score(result, prediction)
            if score > best_score:
                best_score = score
                best_ngram_range = (n, m)


    return best_ngram_range
best_n = select_ngram_range(x_train, y_train, 1, 5)
print(best_n)

(1, 5)


In [ ]:
best_n = (1, 5)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=best_n)
count_vectorised_x_train = vectorizer.fit_transform(x_train)
count_vectorised_x_test = vectorizer.transform(x_test)
bayes_for_count_vectoriser = MultinomialNB().fit(count_vectorised_x_train, y_train)
count_vectoriser_prediction = bayes_for_count_vectoriser.predict(count_vectorised_x_test)

In [ ]:
# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

def select_params(corpus, result, min_n, max_n):
    best_ngram_range = (min_n, min_n)
    best_score = 0
    for n in range(min_n, max_n+1):
        for m in range(n+1, max_n+1):
          for maxdf in np.linspace(0.1, 1, 3):
            for mindf in np.linspace(0.1, 1, 3):
              try:
                  vectorizer = TfidfVectorizer(ngram_range=(n, m), max_df=maxdf, min_df=mindf)
                  X = vectorizer.fit_transform(corpus)
                  bayes = MultinomialNB().fit(X, result)
                  prediction = bayes.predict(X)
                  score = accuracy_score(result, prediction)

              except ValueError:
                  score = 0
              if score > best_score:
                  best_score = score
                  best_ngram_range = (n, m)
                  best_max_df = maxdf
                  best_min_df = mindf

    return best_ngram_range, best_max_df, best_min_df, 10
best_n, max_df, min_df, max_features = select_params(x_train, y_train, 1, 5)

In [ ]:
best_n, max_df, min_df, max_features = (1, 5), 0.65, 0.0, 190

In [ ]:
print(best_n, max_df, min_df, max_features)

(1, 5) 0.65 0.0 190


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=best_n, max_df=max_df, min_df=min_df, max_features=max_features)
tfidf_vectorised_x_train = tfidf_vectorizer.fit_transform(x_train)
tfidf_vectorised_x_test = tfidf_vectorizer.transform(x_test)
bayes_for_tfidf_vectoriser = MultinomialNB().fit(tfidf_vectorised_x_train, y_train)
tfidf_vectoriser_prediction = bayes_for_tfidf_vectoriser.predict(tfidf_vectorised_x_test)

In [ ]:
# Символьная n-грамма

def select_ngram_range(corpus, result, min_n, max_n):
    best_ngram_range = (min_n, min_n)
    best_score = 0
    for n in range(min_n, max_n+1):
        for m in range(n+1, max_n+1):
            vectorizer = CountVectorizer(analyzer='char', ngram_range=(n, m))
            X = vectorizer.fit_transform(corpus)
            bayes = MultinomialNB().fit(X, result)
            prediction = bayes.predict(X)
            score = accuracy_score(result, prediction)
            if score > best_score:
                best_score = score
                best_ngram_range = (n, m)


    return best_ngram_range
best_n = select_ngram_range(x_train, y_train, 1, 5)
best_n = (4, 5)
print(best_n)

(4, 5)


In [ ]:
char_vectorizer = CountVectorizer(analyzer='char', ngram_range=best_n)
char_count_vectorised_x_train = char_vectorizer.fit_transform(x_train)
char_count_vectorised_x_test = char_vectorizer.transform(x_test)
bayes_for_char_count_vectoriser = MultinomialNB().fit(char_count_vectorised_x_train, y_train)
char_count_vectoriser_prediction = bayes_for_char_count_vectoriser.predict(char_count_vectorised_x_test)

In [ ]:
results = pd.DataFrame(columns=['vectoriser_name', 'precision', 'recall', 'f1_score', 'accuracy'])
results.loc[len(results.index)] = [
                'CountVectoriser',
                precision_score(y_test, count_vectoriser_prediction, average='weighted'),
                recall_score(y_test, count_vectoriser_prediction, average='weighted'),
                f1_score(y_test, count_vectoriser_prediction, average='weighted'),
                accuracy_score(y_test, count_vectoriser_prediction)]
results.loc[len(results.index)] = ['TfIdf',
                precision_score(y_test, tfidf_vectoriser_prediction, average='weighted'),
                recall_score(y_test, tfidf_vectoriser_prediction, average='weighted'),
                f1_score(y_test, tfidf_vectoriser_prediction, average='weighted'),
                accuracy_score(y_test, tfidf_vectoriser_prediction)]
results.loc[len(results.index)] = ['CharCountVectoriser',
                precision_score(y_test, char_count_vectoriser_prediction, average='weighted'),
                recall_score(y_test, char_count_vectoriser_prediction, average='weighted'),
                f1_score(y_test, char_count_vectoriser_prediction, average='weighted'),
                accuracy_score(y_test, char_count_vectoriser_prediction)]

print(results)

       vectoriser_name  precision    recall  f1_score  accuracy
0      CountVectoriser   0.716476  0.716899  0.716511  0.716899
1                TfIdf   0.672747  0.661050  0.663318  0.661050
2  CharCountVectoriser   0.712240  0.701678  0.703751  0.701678


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах.

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **а?** - ноль или один символ **а**
* **а+** - один или более символов **а**
* **а\*** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа


In [ ]:
result = re.findall('a?b.', 'aabbсabbcbb')
print(result)

['abb', 'abb', 'bb']


In [ ]:
result = re.findall('a*b.', 'aabbсabbcbb')
print(result)

['aabb', 'abb', 'bb']


In [ ]:
result = re.findall('a+b.', 'aabbсabbcbb')
print(result)

['aabb', 'abb']


Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**:
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc')
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?
Часть входит в уже найденный 'abca'

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
text = "When the Fox hears the Rabbit scream he comes a-runnin', but not to help."
result = re.findall(r'\b[\w-]{2}', re.sub('-', "", text))
print(result)

['Wh', 'th', 'Fo', 'he', 'th', 'Ra', 'sc', 'he', 'co', 'ar', 'bu', 'no', 'to', 'he']


### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie')
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2)
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
text = "Clarice Starling flinched as the first of the heavy steel gates clashed shut behind her and the bolt shot home. Chilton walked slightly ahead, down the green institutional corridor in an atmosphere of Lysol and distant slammings. Starling was angry at herself for letting Chilton put his hand in her purse and briefcase, and she stepped hard on the anger so that she could concentrate. It was all right. She felt her control solid beneath her, like a good gravel bottom in a fast current. "
result = re.split('\. ', text, maxsplit=2)
print(result)

['Clarice Starling flinched as the first of the heavy steel gates clashed shut behind her and the bolt shot home', 'Chilton walked slightly ahead, down the green institutional corridor in an atmosphere of Lysol and distant slammings', 'Starling was angry at herself for letting Chilton put his hand in her purse and briefcase, and she stepped hard on the anger so that she could concentrate. It was all right. She felt her control solid beneath her, like a good gravel bottom in a fast current. ']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
text = "On March 22, 1975, he failed to appear for a performance in Baltimore. On March 25 his body was discovered seated in a pew in a small rural church "
result = re.sub(r'\d', 'DIG', text)
print(result)

On March DIGDIG, DIGDIGDIGDIG, he failed to appear for a performance in Baltimore. On March DIGDIG his body was discovered seated in a pew in a small rural church 


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
text = "Пример URL адреса: https://en.wikipedia.org/wiki/URL#:~:text=Most%20web%20browsers%20display%20the,name%20(%20index.html%20."
result = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+.*[^.]", "", text)
print(result)

Пример URL адреса: .


### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
word_finder = re.compile(r'\b\w{4,}\b')
print(word_finder.findall("Слова? Да, больше, ещё больше слов! Что-то ещё."))

['Слова', 'больше', 'больше', 'слов']


**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
text = "abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz"
adresses = re.compile(r'@\S+(?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b')
adresses.findall(text)

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']